In [1]:
import numpy as np
from math import cos, sin, radians, sqrt, pi, atan, tan


In [2]:
class Elemento():
    def __init__(self,elemento, base, altura, moduloElastico, coordenada_xi, coordenada_yi, coordenada_xf, coordenada_yf, conexion):
        
        self.Elem = elemento
        self.b = base
        self.h = altura
        self.E = moduloElastico
        self.cxi = coordenada_xi
        self.cyi = coordenada_yi
        self.cxf = coordenada_xf
        self.cyf = coordenada_yf
        self.conex = conexion
        # Propiedades del elemento
        self.A = self.Area()
        self.I = self.Inercia()
        self.L = self.Longitud()
        self.ang = self.Angulo()
        self.l = self.coseno()
        self.m = self.seno()

    # Propiedades del elemento

    def Area(self):
        return self.b * self.h
    def Inercia(self):
        return (self.b * self.h**3) / 12
    def Longitud(self):
        return sqrt((self.cxf - self.cxi)**2 + (self.cyf - self.cyi)**2)
    def Angulo(self):
        b = (self.cxf - self.cxi)
        h = (self.cyf - self.cyi)
        try:
            angulo = atan(h/b)*180/pi
        except ZeroDivisionError:
            angulo = 90
        return angulo

    def seno(self):
        return sin(radians(self.ang))

    def coseno(self):
        return cos(radians(self.ang))        



In [3]:

E  = 221.359

tbl_Elem = [
    
  # Elem, Ni, Nf, b, h, E,
   ['1', 1, 2, 30, 30, E],
 ['2', 2, 3, 30, 60, E],
 ['3', 4, 3, 30, 30, E],]
    
tbl_Nod = [
    #Nodo, cx, cy
 [1, 0, 0.0],
 [2, 0, 400.0],
 [3, 1000.0, 400],
 [4, 1000, 0.0],

 ]
        #El, Ni, Nf
#print(nodos[:])
tbl_Fuer = [
    
    #Nod, fx, fy, Mz
    [2,100, 0, 0],
  
    ]

tbl_Fuer_W = [
    #Elem, Li,Lf, Wi, Wf
    #["1", 0, 500, -50, 0], # borrar Li y Lf

    ]

tbl_Rest = [
        #Tipo de Apoyo, Nodo
        # Simple
        # Articulado
        # Empotrado
    ["Empotrado", 1],
    ["Articulado", 4,],
    #["Articulado", 4,],
    ]

"""====Diccionarios===="""
# ==============Diccionarios de elementos==================#

diccionarioE = {}

for i in range(len(tbl_Elem)):
    Elem = tbl_Elem[i][0]
    Ni = tbl_Elem[i][1]
    Nf = tbl_Elem[i][2]
    b = tbl_Elem[i][3]
    h = tbl_Elem[i][4]
    E = tbl_Elem[i][5]   
    diccionarioE[Elem] = [Ni, Nf, b, h, E]

# ==============Diccionarios de nodos==================#
    
diccionarioN = {}

for i in range(len(tbl_Nod)):
    Nod = tbl_Nod[i][0]
    cx = tbl_Nod[i][1]
    cy = tbl_Nod[i][2]
    diccionarioN[Nod] = [cx, cy]
    

In [4]:
elementos = [] # Almacena todos los elementos con sus propiedades

def ext_elem(): # Elementos fuera del analisis

    for i in range(len(tbl_Elem)):
        Ni = tbl_Elem[i][1]
        Nf = tbl_Elem[i][2]
        b = tbl_Elem[i][3]
        h = tbl_Elem[i][4]
        Ele = tbl_Elem[i][0]    
        conexion = [Ele, Ni, Nf]
        xi = diccionarioN[Ni][0]
        yi = diccionarioN[Ni][1]
        xf = diccionarioN[Nf][0]
        yf = diccionarioN[Nf][1]     
        barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
        elementos.append(barra) # Almacena elemento
    return elementos


elementos = ext_elem()
        

In [5]:
sub = 4
coordenadas = []
coor_sub_elementos = []
cheq = []
for i in range(len(tbl_Elem)):
    coor_p = []
    Elemento = elementos[i]
    L = Elemento.L
    L_unitaria = L / sub
    Ni = Elemento.conex[1]
    Nf = Elemento.conex[2]
    cxi = Elemento.cxi
    cyi = Elemento.cyi
    cxf = Elemento.cxf
    cyf = Elemento.cyf
    angulo = Elemento.ang
    cont = 0
    for j in range(sub+1):
        if angulo == 0: 
            cy = Elemento.cyi
            cx = Elemento.cxi + round(L_unitaria*j,5)
            coor_p.append([cx,cy])
            
        if cont < sub:  
            if angulo == 0: 
                cont += 1 
                cy = Elemento.cyi
                cx = Elemento.cxi + round(L_unitaria*j,5)
                cheq.append([cx,cy])
                if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                    coordenadas.append([cx,cy])
                    
        # Elementos verticales
        if angulo == 90:
            cx = Elemento.cxi
            cy = Elemento.cyi + round(L_unitaria*j,5)            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coordenadas.append([cx,cy])
        # Elementos inclinados crecientes
        
        elif angulo > 0 and ang < 90:
            """estos fracmento forman l_uni en un triangulo rectangulo"""
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly_uni/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi + round(Lx_uni*j,5)
            cy = Elemento.cyi + round(Ly_uni*j,5)
            
            #print("j=",j, "Ly= ",Ly_uni,"Lx= ",Lx_uni, "cx= ",cx, "cy= ", cy)
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coordenadas.append([cx,cy])
                
        # Elementos inclinados decrecientes
        elif angulo < 0:
            Ly_uni =  L_unitaria* sqrt(1/(1+(1/(tan(radians(ang))**2)))) # Longitud del fracmento en y
            Lx_uni = Ly/tan(radians(ang)) # Longitud del fracmento en x
            cx = Elemento.cxi - round(Lx_uni*j,5)
            cy = Elemento.cyi - round(Ly_uni*j,5)
            
            cheq.append([cx,cy])
            coor_p.append([cx,cy])
            if cheq.count([cx, cy]) < 2: # Se chequea si la coord ya esta incluida
                coordenadas.append([cx,cy])

    coor_sub_elementos.append(coor_p)
coordenadas
coor_sub_elementos

[[[0, 0.0], [0, 100.0], [0, 200.0], [0, 300.0], [0, 400.0]],
 [[0.0, 400.0],
  [250.0, 400.0],
  [500.0, 400.0],
  [750.0, 400.0],
  [1000.0, 400.0]],
 [[1000, 0.0], [1000, 100.0], [1000, 200.0], [1000, 300.0], [1000, 400.0]]]

In [6]:
tbl_sub_nodos = []
dic_sub_nodos = {}
id_nodo = 0
for n in range(len(coordenadas)):
    cx, cy = coordenadas[n]
    id_nodo += 1
    tbl_sub_nodos.append([id_nodo, cx, cy])
    dic_sub_nodos[id_nodo] = [cx, cy]
tbl_sub_nodos 
dic_sub_nodos

{1: [0, 0.0],
 2: [0, 100.0],
 3: [0, 200.0],
 4: [0, 300.0],
 5: [0, 400.0],
 6: [250.0, 400.0],
 7: [500.0, 400.0],
 8: [750.0, 400.0],
 9: [1000, 0.0],
 10: [1000, 100.0],
 11: [1000, 200.0],
 12: [1000, 300.0],
 13: [1000, 400.0]}

In [7]:
tbl_sub_elementos = []
dic_sub_elementos = {}
id_elementos = 0
for i in range(len(tbl_Elem)):
    Elemento = elementos[i]
    b = elementos[i].b
    h = elementos[i].h
    E = elementos[i].E
    angulo = Elemento.ang
    for j in range(sub):
        id_elementos += 1
        ci = coor_sub_elementos[i][j]
        cf = coor_sub_elementos[i][j+1]
        N = 0
        for m in range(len(tbl_sub_nodos)):
            N = m+1
            if dic_sub_nodos[N] == ci:
                Ni = N
            if dic_sub_nodos[N] == cf:
                Nf = N
        tbl_sub_elementos.append([str(id_elementos), Ni, Nf, b, h, E])
        dic_sub_elementos[str(id_elementos)] = [str(id_elementos), Ni, Nf, b, h, E]
        #print(Ni,Nf)
tbl_sub_elementos           

[['1', 1, 2, 30, 30, 221.359],
 ['2', 2, 3, 30, 30, 221.359],
 ['3', 3, 4, 30, 30, 221.359],
 ['4', 4, 5, 30, 30, 221.359],
 ['5', 5, 6, 30, 60, 221.359],
 ['6', 6, 7, 30, 60, 221.359],
 ['7', 7, 8, 30, 60, 221.359],
 ['8', 8, 13, 30, 60, 221.359],
 ['9', 9, 10, 30, 30, 221.359],
 ['10', 10, 11, 30, 30, 221.359],
 ['11', 11, 12, 30, 30, 221.359],
 ['12', 12, 13, 30, 30, 221.359]]

In [8]:
sub_elementos = [] # Almacena todos los elementos con sus propiedades

for i in range(len(sub_tbl_elementos)):
    Ni = sub_tbl_elementos[i][1]
    Nf = sub_tbl_elementos[i][2]
    b = sub_tbl_elementos[i][3]
    h = sub_tbl_elementos[i][4]
    Ele = sub_tbl_elementos[i][0]    
    conexion = [Ele, Ni, Nf]
    xi = dic_sub_nodos[Ni][0]
    yi = dic_sub_nodos[Ni][1]
    xf = dic_sub_nodos[Nf][0]
    yf = dic_sub_nodos[Nf][1]     
    barra = Elemento(Ele,b, h, E, xi, yi, xf, yf, conexion)
    sub_elementos.append(barra) # Almacena elemento

NameError: name 'sub_tbl_elementos' is not defined

In [154]:
# Obtener cargar externas para los sub elementos
tbl_sub_F_W = []

for i in range(len(tbl_Fuer_W)):
    Elemento = tbl_Fuer_W[i][0]
    wi = tbl_Fuer_W[i][3]
    wf = tbl_Fuer_W[i][4]
    L = 0
    
    for n in range(len(elementos)): # Extra la longitud del elemento sin sub...
        if Elemento == elementos[n].Elem:
            L = elementos[n].L
            
    for j in range(sub):
        Elem = diccionario_sub_E[Elemento][j][0]
        ################################################
        ##################################################
        
        # Carga distibuida rectangular
        #print(wi, wf)
        if wi == wf:
            
            L_unitaria = L/sub*(j+1)
            #print(L_unitaria)
            sub_W = wi
            tbl_sub_F_W.append([Elem, L/sub*j, L_unitaria,sub_W , sub_W])
            
            
        #########################################################
        ##############################################################
            
        # Carga distibuida triangular decreciente
        
        if  abs(wi) > 0  and abs(wf) == 0:
            
            W = wi
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        
        # Carga distibuida triangular creciente
        if  abs(wf) > 0  and abs(wi) == 0:
            
            W = wf
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L 
            sub_wf = (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal decreciente
        if  abs(wi) > abs(wf) and abs(wf) > 0:
            
            W = (wi - wf)
            L_unitaria = L/sub
            sub_wi = (W * L_unitaria * j) /L + wf
            sub_wf = (W * L_unitaria * (j+1))/L + wf
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])
        
        # Carga distibuida trapezoidal creciente
        if  abs(wi) < abs(wf) and abs(wi) > 0:
            
            W = (wf - wi)
            L_unitaria = L/sub
            sub_wi = wi + (W * L_unitaria * j) /L
            sub_wf = wi + (W * L_unitaria * (j+1))/L
            tbl_sub_F_W.append([Elem, 0, L_unitaria,sub_wi , sub_wf])


In [155]:
tbl_sub_restricciones = []

for j in range(len(tbl_Rest)):
    N_rest = tbl_Rest[j][1]
    sub_N_rest = sub * N_rest - (sub-1)
    tipo = tbl_Rest[j][0]
    #print(sub_N_rest)
    for i in range(len(tbl_sub_nodos)):
        sub_N = i+1
        print(sub_N)
        if sub_N == sub_N_rest:
            if sub_N_rest != 1:
                tbl_sub_restricciones.append([tipo, sub_N-sub])
            else:
                tbl_sub_restricciones.append([tipo, sub_N])
        
tbl_sub_restricciones

1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
10
11
12
13


[['Empotrado', 1], ['Articulado', 9]]

In [156]:
tbl_sub_elementos

[['1', 1, 2, 30, 30, 221.359],
 ['2', 2, 3, 30, 30, 221.359],
 ['3', 3, 4, 30, 30, 221.359],
 ['4', 4, 5, 30, 30, 221.359],
 ['5', 5, 6, 30, 60, 221.359],
 ['6', 6, 7, 30, 60, 221.359],
 ['7', 7, 8, 30, 60, 221.359],
 ['8', 8, 13, 30, 60, 221.359],
 ['9', 9, 10, 30, 30, 221.359],
 ['10', 10, 11, 30, 30, 221.359],
 ['11', 11, 12, 30, 30, 221.359],
 ['12', 12, 13, 30, 30, 221.359]]

In [157]:
tbl_sub_nodos

[[1, 0, 0.0],
 [2, 0, 100.0],
 [3, 0, 200.0],
 [4, 0, 300.0],
 [5, 0, 400.0],
 [6, 250.0, 400.0],
 [7, 500.0, 400.0],
 [8, 750.0, 400.0],
 [9, 1000, 0.0],
 [10, 1000, 100.0],
 [11, 1000, 200.0],
 [12, 1000, 300.0],
 [13, 1000, 400.0]]

In [158]:
tbl_sub_restricciones

[['Empotrado', 1], ['Articulado', 9]]